In [1]:
import torch
#from utility import My_Dataset
from utility import mean_and_std,get_params_and_gradients_norm,train_model
from utility import validation, early_stopping
from utility import Augmented_Dataset 
from utility import data_augmentation
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import Dataset #
from torch.utils.data import DataLoader,ConcatDataset
import torchvision.datasets as datasets 
import torchvision.transforms as T
#from torch.utils.tensorboard import SummaryWriter 
import numpy as np #
from skimage import io #
from matplotlib import pyplot as plt

/home/maria/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
import PIL 
import os 
import cv2 
from PIL import Image 
from matplotlib import pyplot as plt

In [3]:
import numpy as np
from skimage import io 

In [4]:
import torchvision.models as models

In [5]:
from torch.utils.tensorboard import SummaryWriter 

In [6]:
transforms = T.Compose([
    T.ToTensor(),
    T.Lambda(lambda x: x.repeat(3,1,1)),
    #T.clone().expand(3, 224, 224),
    #T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #this mean and std is calculated on the dataset with which AlesNet has been trained 
    #T.Normalize([mean], [std]),
    #T.clone().expand(3, 256, 256),
])
#(self,root,t

In [7]:
train_dataset1 = Augmented_Dataset(root="train",transform=transforms,dimension=224,augmentation=data_augmentation,flip=False,crop=True,train=False)#,expand=True)
train_dataset2 = Augmented_Dataset(root="train",transform=transforms,dimension=224,train=False)#,expand=True)
train_dataset3 = Augmented_Dataset(root="train",transform=transforms,dimension=224,augmentation=data_augmentation,flip=True,crop=False,train=False)#,expand=True)
train_dataset4 = Augmented_Dataset(root="train",transform=transforms,dimension=224,augmentation=data_augmentation,flip=True,crop=True,train=False)#,expand=True)
augmented_dataset = ConcatDataset([train_dataset1,train_dataset2,train_dataset3,train_dataset4])
val_dataset = Augmented_Dataset(root="train",transform=transforms,dimension=224,val=True)#,expand=True)

In [25]:
test_dataset = Augmented_Dataset(root="test",transform=transforms,dimension=224)#,expand=True)
train_loader = DataLoader(dataset = augmented_dataset, batch_size=32, shuffle=True)

In [26]:
val_loader = DataLoader(dataset = val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset = test_dataset, batch_size=32, shuffle=True)

In [10]:
len(val_loader)

4

In [11]:
model =  models.alexnet(pretrained=True)

In [12]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [13]:
for param in model.parameters():
    param.requires_grad = False 

In [14]:
model.classifier[6] = nn.Linear(4096,15,bias=True)

In [15]:
#device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([pars for name, pars in model.named_parameters() if "classifier.6" in name],lr =0.01)
#optimizer = optim.Adam(model.parameters(),lr =0.001)
writer = SummaryWriter(f'runs/Prova')
PATH = "prova.py"

In [16]:
_ = model.train() 

In [23]:
def check_accuracy(loader, model): 
    num_correct = 0
    num_samples = 0
    
    model.eval() 
    
    with torch.no_grad(): 
        for x,y in loader: 
            #x = x.to(device=device)
            #y = y.to(device=device)
            #x = x.reshape(x.shape[0],-1)
            
            scores = model(x) 
            
            #s4*10 
            _ , predictions = scores.max(1)
            num_correct += (predictions==y).sum() 
            num_samples += predictions.size(0) 
         
        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        

In [17]:
ep =early_stopping(PATH,1,model,train_loader,val_loader, criterion, optimizer,writer,10,patience=15,device="cpu")

In [19]:
new_model =  models.alexnet(pretrained=True)

In [20]:
new_model.classifier[6] = nn.Linear(4096,15,bias=True)

In [21]:
check = torch.load(PATH)
new_model.load_state_dict(check['parameters'])


<All keys matched successfully>

In [24]:
accuracy = check_accuracy(val_loader, new_model)

Got 225/225 with accuracy 100.00


In [27]:
accuracy = check_accuracy(test_loader, new_model)

Got 2518/2985 with accuracy 84.36


In [33]:
print(new_model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 